In [1]:
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait

## QUAL A IDEIA
+ Entrar no site
+ Pesquisar por Notebook
+ Pegar todos os resultados da primeira página
+ Abrir cada resultado e extrair:
    1. Nome
    2. Preço a vista
    3. Preço a prazo

In [18]:
# Americanas

americanas_df = {
    "name": [],
    "debit_price": [],
    "credit_price": []
}

driver = webdriver.Chrome()
wait = WebDriverWait(driver, poll_frequency=2, timeout=5)

sc = "notebook intel i5 RAM 4gb".replace(" ", "-")
rc = "notebook intel i5 RAM 4gb".replace(" ", "+")

driver.get(f"https://www.americanas.com.br/busca/{sc}?rc={rc}")

element = driver.find_element(By.XPATH, '//*[@id="rsyswpsdk"]/div/main/div/div[3]')
wait.until(lambda a: element.is_displayed())

elms = element.find_elements(By.CLASS_NAME, 'iRvjrG')

for elm in elms:
    try:
        americanas_df["name"].append(elm.find_element(By.CLASS_NAME, 'gUjFDF').get_attribute('innerHTML'))
        americanas_df["debit_price"].append(elm.find_element(By.CLASS_NAME, 'JbUli').get_attribute('innerHTML'))
        americanas_df["credit_price"].append(elm.find_element(By.CLASS_NAME, 'liXDNM').get_attribute('innerHTML'))
    except:
        print("Erro")
        continue

driver.quit()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"tag name","selector":"span"}
  (Session info: chrome=117.0.5938.89); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6C54478A2+54818]
	(No symbol) [0x00007FF6C53B6AD2]
	(No symbol) [0x00007FF6C526DA3B]
	(No symbol) [0x00007FF6C52AE4FC]
	(No symbol) [0x00007FF6C52AE67C]
	(No symbol) [0x00007FF6C52A50FC]
	(No symbol) [0x00007FF6C52CEAEF]
	(No symbol) [0x00007FF6C52A5036]
	(No symbol) [0x00007FF6C52CECC0]
	(No symbol) [0x00007FF6C52E75A2]
	(No symbol) [0x00007FF6C52CE883]
	(No symbol) [0x00007FF6C52A3691]
	(No symbol) [0x00007FF6C52A48D4]
	GetHandleVerifier [0x00007FF6C57AB9A2+3610402]
	GetHandleVerifier [0x00007FF6C5801870+3962352]
	GetHandleVerifier [0x00007FF6C57F9D5F+3930847]
	GetHandleVerifier [0x00007FF6C54E3656+693206]
	(No symbol) [0x00007FF6C53C1638]
	(No sym